In [10]:
import Modules.SQcircuit_extensions as sq_ext
import Modules.figures as figs
import SQcircuit as sq
import numpy as np
import matplotlib.pyplot as plt
import importlib
import qutip as qt

plt.rcParams['text.usetex'] = False
importlib.reload(sq_ext)
importlib.reload(sq)
importlib.reload(figs)
np.set_printoptions(linewidth=200, formatter={'float': '{:.5f}'.format})

<Figure size 640x480 with 0 Axes>

In [11]:
def resonator_C_resonator(Cc, C_val, L_val):
    Cc = sq.Capacitor(Cc, 'fF')
    C = sq.Capacitor(C_val,   'fF')
    L = sq.Inductor (L_val,   'nH')
    elements = {(0, 1): [L, C],
                (0, 2): [L, C],
                (1, 2): [Cc], }
    return sq.Circuit(elements)

def resonator_resonator(C_val, L_val, connected=True ):
    C = sq.Capacitor(C_val, 'fF')
    L = sq.Inductor (L_val,  'nH')
    if connected:
        elements = {(0, 1): [L, C, L, C], }
    else:
        elements = {(0, 1): [L, C],
                    (0, 2): [L, C], }
    return sq.Circuit(elements)

def resonator_resonator_eq(C_val, L_val, connected=True):
    if connected:
        C = sq.Capacitor(C_val+C_val, 'fF')
        L = sq.Inductor ((1/L_val+1/L_val)**-1, 'nH')
        elements = {(0, 1): [L, C], }
    else:
        C = sq.Capacitor(C_val, 'fF')
        L = sq.Inductor (L_val,  'nH')
        elements = {(0, 1): [L, C],
                    (0, 2): [L, C], }
    return sq.Circuit(elements)

def resonator(Cc, C_val, L_val):
    # CC = sq.Capacitor(Cc, 'fF')
    # C = sq.Capacitor(C_val, 'fF')
    # L = sq.Inductor(L_val,  'nH')
    # elements = {(0, 2): [L],
    #             (0, 1): [C],
    #             (1, 2): [CC]}
    # return sq.Circuit(elements)
    
    if Cc == 0:
        C = sq.Capacitor(C_val, 'fF')
        L = sq.Inductor(L_val,  'nH')
        elements = {(0, 1): [L, C], }
        return sq.Circuit(elements)
    else:
        C = sq.Capacitor(1/(1/C_val+1/Cc), 'fF')
        L = sq.Inductor (L_val, 'nH')
        elements = {(0, 1): [L, C], }
    return sq.Circuit(elements)
    
def shutned_resonator(Cc, C_val, L_val):
    Cc = sq.Capacitor(Cc, 'fF')
    C = sq.Capacitor(C_val,   'fF')
    L = sq.Inductor (L_val,   'nH')
    elements = {(0, 1): [L, C, Cc], }
    return sq.Circuit(elements)

In [12]:
fF = 1e-15
nH = 1e-9
C_val = 10
L_val = 1

In [13]:
n_eig = 8
trunc_num = 60
Cc = 5

In [14]:
coupled_res = resonator_C_resonator(Cc=Cc, C_val=C_val, L_val=L_val)
res = resonator(Cc=Cc, C_val=C_val, L_val=L_val)

uncoupled_res = resonator_C_resonator(Cc=0, C_val=C_val, L_val=L_val)
res_0 = resonator(Cc=0, C_val=C_val, L_val=L_val)

uncoupled_renormalized_res = resonator_C_resonator(Cc=0, C_val=(1/C_val+1/Cc)**-1, L_val=L_val)
renormalized_res = resonator(Cc=Cc, C_val=C_val, L_val=L_val)

coupled_res.set_trunc_nums([trunc_num, trunc_num])
uncoupled_res.set_trunc_nums([trunc_num, trunc_num])
uncoupled_renormalized_res.set_trunc_nums([trunc_num, trunc_num])
res.set_trunc_nums([trunc_num  ])
res_0.set_trunc_nums([trunc_num])
renormalized_res.set_trunc_nums([trunc_num])

_ = coupled_res.diag(n_eig)
_ = uncoupled_res.diag(n_eig)
_ = uncoupled_renormalized_res.diag(n_eig)
_ = res.diag(n_eig)
_ = res_0.diag(n_eig)
_ = renormalized_res.diag(n_eig)

In [15]:
Q_0 = res_0._charge_op_isolated(0)
Φ_0 = res_0._flux_op_isolated(0)

Q = res._charge_op_isolated(0)
Φ = res._flux_op_isolated(0)

In [32]:
C_mat = np.array([[C_val, -Cc],
                  [-Cc, C_val]])
C_inv = np.linalg.inv(C_mat)
C_inv

array([[0.13333, 0.06667],
       [0.06667, 0.13333]])

In [31]:

C1_tilde = C_inv[0,0]**-1
C2_tilde = C_inv[1,1]**-1
C_coup = C_inv[0,1]**-1 /3
C_coup, Cc

(5.0, 5)

In [ ]:
C

In [17]:
C_coup = Cc

In [26]:
# H_res_0 = 1 / 2 * (1/fF)               * Q_0 ** 2  + 1 / 2 * (1/nH) * Φ_0 ** 2
H_res_0 = res_0.hamiltonian()
# H_res   = 1 / 2 * (1/fF) * (1 + 1/Cc)  * Q ** 2    + 1 / 2 * (1/nH) * Φ   ** 2# 
H_res   = res.hamiltonian()

I = qt.identity(H_res.shape[0])

H_unc_res = qt.tensor(H_res_0, I) + qt.tensor(I, H_res_0)

H_unc_renorm = qt.tensor(H_res, I) + qt.tensor(I, H_res)

H_coup_res = qt.tensor(H_res, I) + qt.tensor(I, H_res) -  (0.5 / (C_coup*fF)) * qt.tensor(Q, Q)

In [27]:
E = sq_ext.diag(H_coup_res,n_eig=n_eig, out='GHz')[0]
E -= E[0]
E/2

array([0.00000, 35.58813, 50.32921, 71.17625, 85.91734, 100.65842, 106.76438, 121.50547])

In [28]:
# E/E[1]

In [29]:
E = coupled_res.efreqs - coupled_res.efreqs[0]
E

array([0.00000, 35.58813, 50.32921, 71.17625, 85.91734, 100.65842, 106.76438, 121.50547])

In [22]:
# E/E[1]

# Other things

In [23]:
E_0 = sq_ext.diag(H_unc_renorm,n_eig=n_eig, out='GHz')[0]
E_0 - E_0[0]

array([0.00000, 87.17275, 87.17275, 174.34550, 174.34550, 174.34550, 261.51826, 261.51826])

In [100]:
uncoupled_renormalized_res.efreqs - uncoupled_renormalized_res.efreqs[0]

array([0.00000, 177.94064, 177.94064, 355.88127, 355.88127, 355.88127, 533.82191, 533.82191])

In [14]:
import numpy as np
from scipy.linalg import eigh

# Parameters
hbar = 1   # Planck constant (can be set to 1 for simplicity)
C = (1/C_val+1/Cc)**-1 * fF
L = L_val * nH
omega = 1/np.sqrt(L*C) # Resonant frequency of the LC circuits
g = 3/2*np.sqrt(C/L)/(Cc*fF)    # Coupling strength

# Creation and annihilation operators for a single quantum state
def create(n):
    return np.diag(np.sqrt(np.arange(1, n)), -1)

def annihilate(n):
    return np.diag(np.sqrt(np.arange(1, n)), 1)

# Size of the matrix (for 2 quanta in each mode)
n = 3  # Number of states for each mode (0, 1, 2)

# Create operators for each mode
a1 = np.kron(create(n), np.eye(n))
a1_dagger = np.kron(annihilate(n), np.eye(n))
a2 = np.kron(np.eye(n), create(n))
a2_dagger = np.kron(np.eye(n), annihilate(n))

# Construct the Hamiltonian
H = hbar * omega * (np.dot(a1_dagger, a1) + np.dot(a2_dagger, a2)) + hbar * g * (np.dot(a1_dagger, a2) + np.dot(a1, a2_dagger))

# Find the eigenvalues (energy levels)
energies = eigh(H, eigvals_only=True)

# Print the energy levels
E = energies/ 2 / np.pi / 1e9
E -= E[0]
E

array([0.00000, 0.00000, 0.00000, 137.83222, 275.66445, 275.66445, 551.32890, 551.32890, 689.16112])

In [15]:
E/E[1]

array([0.00000, 1.00000, 2.96889, 13968237781629446.00000, 27936475563258892.00000, 27936475563258904.00000, 55872951126517800.00000, 55872951126517808.00000, 69841188908147248.00000])

In [16]:
g/omega

1.0